In [1]:
import pandas as pd

# Beispiel-DataFrame
data = {'A': [1, 1, 1, 1, 1, 1, 2, 3],
        'B': [10, 10, 10, 10, 10, 10, 20, 30]}
df = pd.DataFrame(data)

# Duplikate identifizieren
duplicates = df[df.duplicated(keep=False)]

# Wähle gezielt 4 Zeilen aus den Duplikaten aus, die behalten werden sollen
keep_rows = duplicates.iloc[:4]  # Hier kannst du Kriterien anpassen, z.B. bestimmte Indizes

# Restliche Duplikate entfernen
cleaned_df = pd.concat([df[~df.duplicated(keep=False)], keep_rows]).reset_index(drop=True)

print(cleaned_df)

   A   B
0  2  20
1  3  30
2  1  10
3  1  10
4  1  10
5  1  10


In [2]:
duplicates

,A,B
0,1,10
1,1,10
2,1,10
3,1,10
4,1,10
5,1,10
